## 基本概念介绍：Dataset 类和DataLoader 方法

###### Pytorch提供了一个数据集制作和batch数据读取的工作流程，通过使用：torch.utils.data.Dataset 类和 torch.utils.data.DataLoader 类 。

大概流程是先把原始数据转变成 torch.utils.data.Dataset 类，随后再把得到的 torch.utils.data.Dataset 类当作一个参数传递给 torch.utils.data.DataLoader 类，得到一个数据加载器，这个数据加载器每次可以返回一个 Batch 的数据供模型训练使用。

### Dataset类
torch.utils.data.Dataset 的源码如下：

In [3]:
class Dataset(object):
    """An abstract class representing a Dataset.

    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

#### 为什么需要通过继承Dataset类来生成数据集？
#### 主要原因是可以实现自定义读取数据集，如通过重写\__init__()，可以实现直接给类传入链接而不是Tensor就生成数据集，相当于把一些数据的前期处理过程代码直接写到类里，方便以后直接调用。

Dataset是一个抽象类，抽象类的定义是：必须要对这个类进行继承和重写部分功能之后才能使用。   
    
根据注释，需要重写的包含两个功能，一个是通过索引和切片返回数据的功能\__getitem__()，即当对实例使用索引和切片时将调用的功能，假如实例是x，x[i]或者x[i:j]都将执行\__getitem__()的具体内容，\__getitem__(self, index)包含一个需要传入的参数，这个参数可以是索引的编号，也可以是切片，如x[2]或x[2:4]的形式都可以。
     
另一个需要重写的功能是\__len__(self)，即对实例调用.len()方法时执行\__len__(self)里的具体内容，返回数据集的长度。
     
最后，一般还需要自己重写\__init__()方法
    
示例如下：

In [36]:
from torch.utils.data import Dataset
import torch

class TensorDataset(Dataset):
    # TensorDataset继承Dataset, 重载了__init__, __getitem__, __len__
    # 实现将一组Tensor数据对封装成Tensor数据集
    # 能够通过index得到数据集的数据，能够通过len，得到数据集大小

    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.data_tensor.size(0)    # size(0) 返回当前张量维数的第一维

# 生成数据
data_tensor = torch.randn(4, 3)   # 4 行 3 列，服从正态分布的张量
print(data_tensor)
target_tensor = torch.rand(4)     # 4 个元素，服从均匀分布的张量
print(target_tensor)

# 将数据封装成 Dataset （用 TensorDataset 类）
tensor_dataset = TensorDataset(data_tensor, target_tensor)
print('tensor_dataset type:',type(tensor_dataset))

# 可使用索引调用数据
print('tensor_data[0]: ', tensor_dataset[0])

# 可返回数据len
print('len os tensor_dataset: ', len(tensor_dataset))


tensor([[ 1.4013,  0.2007, -0.7327],
        [ 0.1431, -0.2383,  0.4414],
        [-0.2592, -0.0935, -1.4246],
        [-1.0566,  1.0683,  1.7570]])
tensor([0.6143, 0.8208, 0.4959, 0.2809])
tensor_dataset type: <class '__main__.TensorDataset'>
tensor_data[0]:  (tensor([ 1.4013,  0.2007, -0.7327]), tensor(0.6143))
len os tensor_dataset:  4


### TensorDataset类：
#### 如果训练数据和测试数据已经是Tensor数据了，不需要进行额外的数据读取和处理，有一个现成的类TensorDataset可以直接将训练和测试数据组合成一个整体，不需要继承和重写Dataset类。实际上TensorDataset是torch官方写好的一个Dataset子类
可以传入多个tensor(如训练数据和label可以作为两个tensor一起传入,也可以传入多个Tensor，下面的例子就传入了三个Tensor)，返回一个对象，这个对象将按照传入数据的第0个维度作为索引范围，根据索引输出数据。

In [10]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

a=torch.randn(10,4)
b=torch.randn(10).round()
c=torch.randn(10,4)

print(a)
print(b,'\n')
print(c,'\n')

ds = TensorDataset(a,b,c)
print('索引为2的数据为：\n',ds[2])

tensor([[ 0.2144,  0.5958, -0.9231, -0.0603],
        [ 0.0452, -0.0901,  0.9440, -0.8051],
        [ 1.6106,  2.4077,  0.0359,  1.5005],
        [-0.4482,  0.6982, -2.0945, -1.0440],
        [ 0.0654,  0.5342,  0.6443, -0.9608],
        [ 1.1128,  1.4329, -0.1786,  0.5952],
        [-0.3398,  0.8663, -1.9183,  2.0390],
        [-0.3097,  1.4683, -0.9541, -0.1568],
        [ 0.7137, -0.2179,  0.8163,  1.3313],
        [ 0.2613,  0.1538, -1.6363,  0.4333]])
tensor([ 0.,  1.,  1., -0., -1., -1.,  1.,  2.,  2., -1.]) 

tensor([[-2.0350, -1.3459, -0.7661, -0.2537],
        [ 2.0352,  0.6913, -1.1180, -0.7018],
        [ 0.0879,  0.3121,  0.4109,  0.6585],
        [ 1.0450,  1.0926,  1.1654, -0.2696],
        [ 1.0059,  1.6081, -0.5125,  0.1624],
        [-1.1278,  0.4157,  0.6712, -0.1647],
        [-0.0264, -2.0133,  0.4253,  0.4087],
        [-1.2524, -0.0863, -0.4273, -0.0268],
        [-2.2537, -0.1405, -0.3141, -0.0217],
        [-0.6898, -0.3003,  0.3828,  1.1826]]) 

索引为2的数据为：
 (ten

### DataLoader类

#### DataLoader类，返回一个类似的可迭代实例。但是不能用next()，也不能用索引和切片，只能用for循环读取。     
#### 重要参数包括:
drop_last = False 是否丢弃最后未能整除的batch数据    
batch_size = 10 每一个batch的大小    
shuffle = False 是否乱序

```python
DataLoader(
    Dataset  #这个Dataset参数传入的内容需要是torch.utils.data.dataset.Dataset类的实例对象。可以是自己继承于Dataset抽象类的子类的实例；也可以是torch中官方写的继承于Dataset抽象类的子类的实例，如TensorData类的实例
    batch_size=1,   #批次大小
    shuffle=False,  #是否乱序
    sampler=None,   #样本采样函数，一般无需设置。
    batch_sampler=None,   #批次采样函数，一般无需设置。
    num_workers=0,  #使用多进程读取数据，设置的进程数。
    collate_fn=None, #整理一个批次数据的函数。
    pin_memory=False,#是否设置为锁业内存。默认为False，锁业内存不会使用虚拟内存(硬盘)，从锁业内存拷贝到GPU上速度会更快。
    drop_last=False, #是否丢弃最后一个样本数量不足batch_size批次数据。
    timeout=0,     #加载一个数据批次的最长等待时间，一般无需设置。
    worker_init_fn=None, #每个worker中dataset的初始化函数，常用于 IterableDataset。一般不使用。
    multiprocessing_context=None,
)
```


#### 用索引读取返回的DataLoader报错

In [34]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

a=torch.randn(7)
b=torch.randn(7).round()

ds = TensorDataset(a,b)
print(type(ds))
dl=DataLoader(ds,batch_size=4,shuffle=False,drop_last=False)
#DataLoader返回的是一个类list对象，需要通过for循环进行读取

dl[1]

<class 'torch.utils.data.dataset.TensorDataset'>


TypeError: 'DataLoader' object is not subscriptable

#### 用for循环读取不报错

In [32]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

a=torch.randn(7)
b=torch.randn(7).round()

ds = TensorDataset(a,b)
dl=DataLoader(ds,batch_size=4,shuffle=False,drop_last=False)
#DataLoader返回的是一个类list对象，需要通过for循环进行读取

for i in dl:
    print(i)

[tensor([-1.1200,  2.4612,  0.8978, -0.1737]), tensor([2., 1., 1., 0.])]
[tensor([2.1077, 0.0515, 0.7662]), tensor([ 0., -1., -1.])]


#### 使用枚举和元组可以将每一条的多个数据进行分别提取

In [33]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

a=torch.randn(7)
b=torch.randn(7).round()

ds = TensorDataset(a,b)
dl=DataLoader(ds,batch_size=4,shuffle=False,drop_last=False)
#DataLoader返回的是一个类list对象，需要通过for循环进行读取

for i,(batch,label) in enumerate(dl):
    print(f'batch{i}')
    print('train:',batch,)
    print('label:',label,'\n\n')

batch0
train: tensor([ 1.1065,  0.1573,  1.4133, -0.4462])
label: tensor([-0., -1., -2.,  0.]) 


batch1
train: tensor([0.3775, 0.0796, 2.0775])
label: tensor([0., 0., -0.]) 


